# Understanding Multi-Objective Genetic Algorithms (MOGA)

Multi-Objective Genetic Algorithms (MOGA) represent a class of optimization algorithms inspired by the process of natural selection, a fundamental mechanism of evolution. These algorithms are designed to solve problems involving multiple objectives that often conflict with each other, necessitating a balance to find an optimal set of solutions known as Pareto optimal solutions. Unlike single-objective optimization, where the goal is to find a single best solution, multi-objective optimization acknowledges the trade-offs between competing objectives and seeks a set of solutions that represent the best possible compromises.

MOGAs operate through a population of potential solutions, each represented by a set of variables or "genes". These solutions evolve over generations, driven by operations mimicking natural genetic processes: selection, crossover (or recombination), and mutation. The selection process favors individuals with better fitness scores, allowing them to pass their genes to the next generation. Crossover combines the genetic material of parent solutions to produce offspring, introducing new solution candidates. Mutation introduces random changes to individual genes, preventing premature convergence on suboptimal solutions and ensuring diversity within the population.

The evaluation of solutions in MOGAs involves calculating multiple objective functions for each individual in the population. This multi-dimensional evaluation complicates the selection process, as there is no single fitness value to guide the selection. Instead, MOGAs employ techniques like Pareto dominance, where solutions are ranked based on how many other solutions they dominate (i.e., are better than across all objectives). This approach helps in maintaining a diverse set of solutions that are all, in some sense, optimal.

A key outcome of MOGA is the Pareto front, a set of non-dominated solutions that represent the best trade-offs among the objectives. Identifying the Pareto front helps decision-makers understand the possible compromises and select a solution that best meets their specific needs or preferences. This is particularly valuable in fields like engineering, economics, and environmental management, where multiple objectives must be balanced.

In summary, Multi-Objective Genetic Algorithms are powerful tools for solving complex optimization problems with multiple objectives. They provide a framework for exploring the trade-offs between conflicting goals and identifying a range of optimal solutions, offering flexibility and insight that is not possible with single-objective optimization methods.

**Decision Variables:**
- $x_{\text{res}}$: Area of residential zones.
- $x_{\text{com}}$: Area of commercial zones.
- $x_{\text{ind}}$: Area of industrial zones.
- $x_{\text{park}}$: Area of parks and green spaces.

**Parameters:**
- $A_{\text{total}}$: Total available land area.
- $d_{ij}$: Distance between residential area $i$ and industrial area $j$.
- $D_{\text{min}}$: Minimum acceptable distance between residential and industrial zones to minimize noise pollution.

**Objective Functions:**
1. Maximize Green Space: 
   $$Z_1 = x_{\text{park}}$$
2. Maximize Economic Viability: 
   $$Z_2 = x_{\text{com}} + x_{\text{ind}}$$
3. Minimize Noise Pollution: 
   $$Z_3 = \sum_{i,j} \frac{1}{d_{ij}} \text{ for all residential-industrial pairs, where } d_{ij} < D_{\text{min}}$$

**Constraints:**
1. Total Land Area: 
   $$x_{\text{res}} + x_{\text{com}} + x_{\text{ind}} + x_{\text{park}} = A_{\text{total}}$$
2. Minimum Distance: 
   $$d_{ij} \geq D_{\text{min}} \text{ for all residential-industrial pairs to minimize noise pollution}$$
3. Non-negativity and Bounds: 
   $$x_{\text{res}}, x_{\text{com}}, x_{\text{ind}}, x_{\text{park}} \geq 0$$
   $$x_{\text{res}}, x_{\text{com}}, x_{\text{ind}}, x_{\text{park}} \leq A_{\text{total}}$$


In [1]:
import random
from deap import base, creator, tools, algorithms
import gurobipy as gp
from gurobipy import GRB

# Problem constants
TOTAL_AREA = 1000  # Example total area
MIN_AREA = 50      # Minimum area for each zone
MAX_AREA = 300     # Maximum area for each zone
DIST_THRESHOLD = 50  # Minimum distance between residential and industrial zones

# Define objectives
def maximize_green_space(individual):
    return sum(individual[0:2])  # Assuming first two zones are parks

def maximize_economic_viability(individual):
    return sum(individual[2:4])  # Assuming next two zones are commercial and industrial

def minimize_noise_pollution(individual):
    # Example noise pollution calculation
    # Modify this based on your actual model for noise pollution
    noise_pollution = 0
    for i in range(len(individual)):
        noise_pollution += individual[i] / (i + 1)  # Simplified example
    return noise_pollution

# Create types
creator.create("FitnessMulti", base.Fitness, weights=(1.0, 1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Initialization
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, MIN_AREA, MAX_AREA)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=4)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operators
def evaluate(individual):
    return (maximize_green_space(individual), maximize_economic_viability(individual), minimize_noise_pollution(individual))

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selNSGA2)

# Genetic Algorithm
def run_ga():
    population = toolbox.population(n=50)
    NGEN = 40
    for gen in range(NGEN):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        population = toolbox.select(offspring, k=len(population))
    return population

# Run GA
final_population = run_ga()

# Optimization with Gurobi
def optimize_with_gurobi(individual):
    model = gp.Model("UrbanPlanning")
    vars = model.addVars(len(individual), lb=MIN_AREA, ub=MAX_AREA, vtype=GRB.CONTINUOUS, name="x")
    
    # Objective: Adjust this based on your specific optimization needs
    model.setObjective(gp.quicksum(vars[i] for i in range(len(individual))), GRB.MAXIMIZE)

    # Constraint: Total area
    model.addConstr(gp.quicksum(vars[i] for i in range(len(individual))) <= TOTAL_AREA, "TotalArea")

    model.optimize()
    return model

# Optimize the best solution from GA
best_solution = tools.selBest(final_population, k=1)[0]
optimized_model = optimize_with_gurobi(best_solution)

# Print the optimization report
print("Optimization Runtime:", optimized_model.Runtime)
print("Objective Value:", optimized_model.ObjVal)
for v in optimized_model.getVars():
    print(f'{v.VarName}: {v.X}')


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xad228399
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+01, 3e+02]
  RHS range        [1e+03, 1e+03]
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.000000000e+03
Optimization Runtime: 0.007596015930175781
Objective Value: 1000.0
x[0]: 100.0
x[1]: 300.0
x[2]: 300.0
x[3]: 300.0
